# Fedbiomed Researcher

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

Loading tensorboard extension

In [2]:
%load_ext tensorboard

## Setting the client up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the client
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Connected with result code 0`. it means you are online.

## Create an experiment to train a model on the data found

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [3]:
from fedbiomed.researcher.environ import TMP_DIR
import tempfile
tmp_dir_model = tempfile.TemporaryDirectory(dir=TMP_DIR+'/')
model_file = tmp_dir_model.name + '/class_export_mnist.py'

Note : write **only** the code to export in the following cell

In [4]:
%%writefile "$model_file"

import torch
import torch.nn as nn
from fedbiomed.common.torchnn import TorchTrainingPlan
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self):
        super(MyTrainingPlan, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms",
               "from torch.utils.data import DataLoader"]
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        data_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
        return data_loader
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


Writing /user/scansiz/home/Desktop/Inria/dev/fedbiomed/var/tmp/tmp_4t_foal/class_export_mnist.py


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the client side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the client side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [5]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 3, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

Define an experiment
- search nodes serving data for these `tags`, optionally filter on a list of client ID with `clients`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `rounds` rounds, applying the `client_selection_strategy` between the rounds

In [6]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 #clients=None,
                 model_path=model_file,
                 model_args=model_args,
                 model_class='MyTrainingPlan',
                 training_args=training_args,
                 rounds=rounds,
                 aggregator=FedAverage(),
                 client_selection_strategy=None,
                 tensorboard=True
                )

2021-09-29 14:32:50,976 fedbiomed INFO - Messaging researcher_46a0da2b-2e99-4316-b964-b467d343432b successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f56f2b7a640>
2021-09-29 14:32:50,988 fedbiomed INFO - Searching for clients with data tags: ['#MNIST', '#dataset']
2021-09-29 14:32:50,992 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3'}], 'count': 1, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'command': 'search'}
2021-09-29 14:32:50,994 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'

Let's start the experiment.

By default, this function doesn't stop until all the `rounds` are done for all the clients

In [7]:
exp.run()

2021-09-29 14:33:01,393 fedbiomed INFO - Sampled clients in round 0 ['client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137']
2021-09-29 14:33:01,394 fedbiomed INFO - Send message to client client_2a34fb5d-7c54-469f-b034-51062b388dc1 - {'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': 'f8f3cf36-8457-4d27-939e-e55b0d2c83bc', 'training_args': {'batch_size': 48, 'lr': 0.001, 'epochs': 3, 'dry_run': False, 'batch_maxnum': 100}, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2021/09/29/my_model_a0afef44-3e26-4b89-baef-61df02dc4abd.py', 'params_url': 'http://localhost:8844/media/uploads/2021/09/29/my_model_4c67ef20-2237-4cd5-a1db-e65f446b9f96.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'client_2a34fb5d-7c54-469f-b034-51062b388dc1': ['dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3']}}
2021-09-29 14:33:01,395 fedbiomed DEBUG - researcher_46a0da2b-2e99-4316-b964-b467d343432b
2021

2021-09-29 14:33:21,142 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 3 [960/60000]	Loss: 0.082104
2021-09-29 14:33:21,759 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 3 [1440/60000]	Loss: 0.267302
2021-09-29 14:33:21,908 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 3 [1440/60000]	Loss: 0.157372
2021-09-29 14:33:22,509 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 3 [1920/60000]	Loss: 0.186177
2021-09-29 14:33:22,733 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 3 [1920/60000]	Loss: 0.188419
2021-09-29 14:33:23,270 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 3 [2400/60000]	Loss: 0.192168
2021-09-29 14:33:23,473 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 3 [2400/60000]	L

2021-09-29 14:33:45,157 fedbiomed INFO - Round: 1 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [3840/60000]	Loss: 0.289558
2021-09-29 14:33:45,562 fedbiomed INFO - Round: 1 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [4320/60000]	Loss: 0.129801
2021-09-29 14:33:45,897 fedbiomed INFO - Round: 1 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [4320/60000]	Loss: 0.119760
2021-09-29 14:33:46,476 fedbiomed INFO - Round: 1 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [0/60000]	Loss: 0.269717
2021-09-29 14:33:46,791 fedbiomed INFO - Round: 1 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [0/60000]	Loss: 0.223807
2021-09-29 14:33:47,211 fedbiomed INFO - Round: 1 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [480/60000]	Loss: 0.026569
2021-09-29 14:33:47,574 fedbiomed INFO - Round: 1 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [480/60000]	Loss: 0.

2021-09-29 14:35:33,440 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'success': True, 'databases': [{'name': 'MNIST', 'data_type': 'default', 'tags': ['#MNIST', '#dataset'], 'description': 'MNIST database', 'shape': [60000, 1, 28, 28], 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3'}], 'count': 1, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'command': 'search'}
2021-09-29 14:35:43,935 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [0/60000]	Loss: 2.326869
2021-09-29 14:35:43,954 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [0/60000]	Loss: 2.267578
2021-09-29 14:35:45,984 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [480/60000]	Loss: 1.156422
2021-09-29 14:35:46,274 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [480

2021-09-29 14:36:13,760 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [1440/60000]	Loss: 0.158115
2021-09-29 14:36:14,455 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [1920/60000]	Loss: 0.095185
2021-09-29 14:36:14,679 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [1920/60000]	Loss: 0.163130
2021-09-29 14:36:15,335 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [2400/60000]	Loss: 0.148225
2021-09-29 14:36:15,594 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [2400/60000]	Loss: 0.136248
2021-09-29 14:36:16,173 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [2880/60000]	Loss: 0.129917
2021-09-29 14:36:16,472 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [2880/60000]	

2021-09-29 14:36:51,709 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [3840/60000]	Loss: 0.289804
2021-09-29 14:36:51,902 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 1 [4320/60000]	Loss: 0.024883
2021-09-29 14:36:52,661 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 1 [4320/60000]	Loss: 0.244866
2021-09-29 14:36:52,929 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [0/60000]	Loss: 0.056463
2021-09-29 14:36:53,857 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [0/60000]	Loss: 0.080561
2021-09-29 14:36:53,984 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [480/60000]	Loss: 0.040271
2021-09-29 14:36:55,184 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [480/60000]	Loss: 0.

2021-09-29 14:37:25,688 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [1440/60000]	Loss: 0.089797
2021-09-29 14:37:26,198 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [1920/60000]	Loss: 0.059025
2021-09-29 14:37:27,283 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [1920/60000]	Loss: 0.154550
2021-09-29 14:37:27,787 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [2400/60000]	Loss: 0.148539
2021-09-29 14:37:28,252 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [2400/60000]	Loss: 0.268975
2021-09-29 14:37:28,653 fedbiomed INFO - Round: 0 Node: client_b1afd615-e4d4-489f-9ded-bd8db8ff2137 - Train Epoch: 2 [2880/60000]	Loss: 0.028786
2021-09-29 14:37:29,184 fedbiomed INFO - Round: 0 Node: client_2a34fb5d-7c54-469f-b034-51062b388dc1 - Train Epoch: 2 [2880/60000]	

2021-09-29 14:39:06,988 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '1d777934-84c2-41d8-98f9-f049b85a99cc', 'success': True, 'client_id': 'client_2a34fb5d-7c54-469f-b034-51062b388dc1', 'dataset_id': 'dataset_655ec19c-3a4c-4b23-9e39-c91deba0a3e3', 'params_url': 'http://localhost:8844/media/uploads/2021/09/29/node_params_86900c65-cca7-4c15-b116-baf03b9aeb72.pt', 'timing': {'rtime_training': 27.117097103968263, 'ptime_training': 99.3691229279998}, 'msg': '', 'command': 'train'}
2021-09-29 14:39:46,201 fedbiomed INFO - message received:{'researcher_id': 'researcher_46a0da2b-2e99-4316-b964-b467d343432b', 'job_id': '1d777934-84c2-41d8-98f9-f049b85a99cc', 'success': True, 'client_id': 'client_b1afd615-e4d4-489f-9ded-bd8db8ff2137', 'dataset_id': 'dataset_da9a5db6-9dda-475e-b920-43f8d1f54377', 'params_url': 'http://localhost:8844/media/uploads/2021/09/29/node_params_c74b4fe5-c362-4761-ba28-0862b806193b.pt', 'timing': {'rtime_t

To display current values please click refresh button on the TensorBoard screen

In [ ]:
%tensorboard --logdir '../var/tmp/tensorboard'

Local training results for each round and each node are available in `exp.training_replies` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies.keys())

print("\nList the clients for the last training round and their timings : ")
round_data = exp.training_replies[rounds - 1].data
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['client_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies[rounds - 1].dataframe

Federated parameters for each round are available in `exp.aggregated_params` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params.keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params[rounds - 1]['params'].keys())


## Optional : searching the data

In [ ]:
from fedbiomed.researcher.requests import Requests

r = Requests()
data = r.search(tags)

import pandas as pd
for client_id in data.keys():
    print('\n','Data for ', client_id, '\n\n', pd.DataFrame(data[client_id]))

## Optional : clean file repository (do not run unless necessary)
Clean all the files in the repo via the rest API.

In [ ]:
# import requests
# from fedbiomed.researcher.environ import UPLOADS_URL

# uploaded_models = requests.get(UPLOADS_URL).json()
# for m in uploaded_models:
#   requests.delete(m['url'])

Feel free to try your own models :D